In [ ]:
!wget http://ufldl.stanford.edu/housenumbers/train_32x32.mat
!wget http://ufldl.stanford.edu/housenumbers/test_32x32.mat

--2021-03-21 20:49:31--  http://ufldl.stanford.edu/housenumbers/train_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182040794 (174M) [text/plain]
Saving to: ‘train_32x32.mat’

train_32x32.mat     100%[===================>] 173.61M  34.2MB/s    in 7.9s    

2021-03-21 20:49:39 (21.9 MB/s) - ‘train_32x32.mat’ saved [182040794/182040794]

--2021-03-21 20:49:39--  http://ufldl.stanford.edu/housenumbers/test_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64275384 (61M) [text/plain]
Saving to: ‘test_32x32.mat’

test_32x32.mat      100%[===================>]  61.30M  7.27MB/s    in 11s     

2021-03-21 20:49:50 (5.50 MB/s) - ‘test_32x32.mat’ saved [64275384/64

In [ ]:
import keras
from keras.layers.normalization import BatchNormalization
import numpy as np
import scipy.io as io
from keras.layers import Activation, Flatten,Softmax
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.models import Sequential
from keras import optimizers
from keras.backend import function
import pandas as pd

In [ ]:
from keras.backend import function

In [ ]:
#importing data
train_set = io.loadmat("train_32x32.mat")
test_set = io.loadmat("test_32x32.mat")
x_train = train_set['X']
y_train = train_set['y']
x_test = test_set['X']
y_test = test_set['y']
# Reshaping test and training data sets
x_train = np.moveaxis(x_train,[3,0,1,2],[0,1,2,3])
x_train = x_train.astype('float32')
x_test = np.moveaxis(x_test,[3,0,1,2],[0,1,2,3])
x_test = np.asarray(x_test)
x_test = x_test.astype('float32')
#Normalisation des données
x_train /= x_train.max()
x_test /= x_test.max()
# Passage de classe comme integer à classe comme vecteur de dimension classes
y_test_RN = [y-1 for y in y_test]# la classe 1 corresponds au array qui à 1 en indice 0
y_train_RN = [y-1 for y in y_train]
classes = 10
y_train_RN = keras.utils.to_categorical(y_train_RN, classes)
y_test_RN = keras.utils.to_categorical(y_test_RN, classes)
 #Définition du type de data pour optimiser la mémoire
y_train_RN = y_train_RN.astype('int32')
y_test_RN = y_test_RN.astype('int32')
# Featurewise normalization
datagen = keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    horizontal_flip=False,
    zca_whitening=False, 
    zca_epsilon=1e-06)

datagen.fit(x_train)

In [ ]:
def deepnet2():
  model=Sequential()
  model.add(Conv2D(64, kernel_size=(3, 3),padding = 'same', activation='relu', input_shape=x_train.shape[1:]))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),  padding = 'valid', name='S1'))
  model.add(Conv2D(128, kernel_size=(3, 3), padding = 'valid', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),  padding = 'same', name='S2'))
  model.add(Conv2D(128, kernel_size=(3, 3),  strides = (1,1), padding = 'valid',  activation='relu'))
  model.add(Conv2D(128, kernel_size=(3, 3),  strides = (1,1), padding = 'valid',  activation='relu'))
  model.add(Conv2D(128, kernel_size=(3, 3),  strides = (1,1), padding = 'valid',  activation='relu'))
  model.add(Dropout(0.5)) 
  model.add(Flatten())
  model.add(Dense(classes, activation='softmax'))
  adam = optimizers.Adam(lr = 0.001)
  model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer= adam,
                  metrics=['accuracy'])
  return model

In [ ]:
#model = keras.models.load_model('/content/drive/My Drive/Projet 3A')

In [ ]:
model=deepnet2()

In [ ]:

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
S1 (MaxPooling2D)            (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
S2 (MaxPooling2D)            (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 5, 128)         147584    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 128)         147584    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 1, 1, 128)         1

In [ ]:
model.fit(x_train,y_train_RN, validation_split=0.2 ,epochs=10, batch_size=128)

Epoch 1/10
458/458 [==============================] - 272s 592ms/step - loss: 1.9127 - accuracy: 0.3229 - val_loss: 0.5850 - val_accuracy: 0.8201
Epoch 2/10
458/458 [==============================] - 277s 605ms/step - loss: 0.6170 - accuracy: 0.8181 - val_loss: 0.4178 - val_accuracy: 0.8765
Epoch 3/10
458/458 [==============================] - 277s 604ms/step - loss: 0.4444 - accuracy: 0.8722 - val_loss: 0.3610 - val_accuracy: 0.8943
Epoch 4/10
458/458 [==============================] - 277s 605ms/step - loss: 0.3840 - accuracy: 0.8880 - val_loss: 0.3190 - val_accuracy: 0.9098
Epoch 5/10
458/458 [==============================] - 277s 606ms/step - loss: 0.3237 - accuracy: 0.9067 - val_loss: 0.3058 - val_accuracy: 0.9151
Epoch 6/10
458/458 [==============================] - 276s 602ms/step - loss: 0.2917 - accuracy: 0.9172 - val_loss: 0.2930 - val_accuracy: 0.9163
Epoch 7/10
458/458 [==============================] - 275s 600ms/step - loss: 0.2730 - accuracy: 0.9225 - val_loss: 0.3068 -

In [ ]:
"""
from google.colab import drive
drive.mount('drive')
"""

"\nfrom google.colab import drive\ndrive.mount('drive')\n"

In [ ]:
"""
model_json2 = model2.to_json()
with open("/content/drive/MyDrive/Projet3A/model2.json", "w") as json_file:
    json_file.write(model_json2)
# serialize weights to HDF5
model2.save_weights("model2.h5")
print("Saved model2 to disk")
"""

'\nmodel_json2 = model2.to_json()\nwith open("/content/drive/MyDrive/Projet3A/model2.json", "w") as json_file:\n    json_file.write(model_json2)\n# serialize weights to HDF5\nmodel2.save_weights("model2.h5")\nprint("Saved model2 to disk")\n'

In [ ]:
"""
from keras.models import model_from_json
json_file = open('/content/drive/MyDrive/Projet3A/model2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
"""

"\nfrom keras.models import model_from_json\njson_file = open('/content/drive/MyDrive/Projet3A/model2.json', 'r')\nloaded_model_json = json_file.read()\njson_file.close()\nmodel = model_from_json(loaded_model_json)\n"

In [ ]:
#model2.save('/content/drive/My Drive/Projet 3A')

In [ ]:
L=15000

In [ ]:
scores=[]
x_train_10000=x_train[:L]

In [ ]:
y_test_KNN=[k.item() for k in y_test]
y_train_KNN=[k.item() for k in y_train]

In [ ]:
m=5

# Couche conv2d 50

In [ ]:
model.layers[6]

In [ ]:
get_conv50_output = function([model.layers[0].input],
                                  [model.layers[6].output])
layer_output_conv50 = get_conv50_output([x_train_10000])

In [ ]:
np.array(layer_output_conv50[0]).shape

(15000, 1, 1, 128)

In [ ]:
conv50=np.array(layer_output_conv50[0]).reshape(L,128)

In [ ]:
conv50.shape

(15000, 128)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_conv50=KNeighborsClassifier(m)
knn_conv50.fit(conv50,y_train_KNN[:L])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
layer_output_conv50_test = get_conv50_output([x_test[:5000]])

In [ ]:
conv50_test=np.array(layer_output_conv50_test[0]).reshape(5000,128)

In [ ]:
score=knn_conv50.score(conv50_test,y_test_KNN[:5000])

In [ ]:
scores.append(score)

In [ ]:
scores

[0.9152]

# Conv2D 40

In [ ]:
model.layers[5]

In [ ]:
get_conv40_output = function([model.layers[0].input],
                                  [model.layers[5].output])
layer_output_conv40 = get_conv40_output([x_train_10000])

In [ ]:
np.array(layer_output_conv40[0]).shape

(15000, 3, 3, 128)

In [ ]:
conv40=np.array(layer_output_conv40[0]).reshape(L,1152)

In [ ]:
knn_conv40=KNeighborsClassifier(m)
knn_conv40.fit(conv40,y_train_KNN[:L])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
layer_output_conv40_test = get_conv40_output([x_test[:5000]])

In [ ]:
conv40_test=np.array(layer_output_conv40_test[0]).reshape(5000,1152)

In [ ]:
score=knn_conv40.score(conv40_test,y_test_KNN[:5000])

In [ ]:
scores.append(score)

In [ ]:
scores

[0.9152, 0.8946]

# Conv2D 30

In [ ]:
model.layers[4]

In [ ]:
get_conv30_output = function([model.layers[0].input],
                                  [model.layers[4].output])
layer_output_conv30 = get_conv30_output([x_train_10000])

In [ ]:
np.array(layer_output_conv30[0]).shape

(15000, 5, 5, 128)

In [ ]:
conv30=np.array(layer_output_conv30[0]).reshape(L,3200)

In [ ]:
knn_conv30=KNeighborsClassifier(m)
knn_conv30.fit(conv30,y_train_KNN[:L])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
layer_output_conv30_test = get_conv30_output([x_test[:5000]])

In [ ]:
conv30_test=np.array(layer_output_conv30_test[0]).reshape(5000,3200)

In [ ]:
score=knn_conv30.score(conv30_test,y_test_KNN[:5000])

In [ ]:
scores.append(score)
scores

[0.9152, 0.8946, 0.825]

#Maxpooling S2

In [ ]:
model.layers[3]

In [ ]:
get_S2_output = function([model.layers[0].input],
                                  [model.layers[3].output])
layer_output_S2 = get_S2_output([x_train_10000])

In [ ]:
np.array(layer_output_S2[0]).shape

(15000, 7, 7, 128)

In [ ]:
S2=np.array(layer_output_S2[0]).reshape(L,6272)

In [ ]:
knn_S2=KNeighborsClassifier(m)
knn_S2.fit(S2,y_train_KNN[:L])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
layer_output_S2_test = get_S2_output([x_test[:5000]])

In [ ]:
S2_test=np.array(layer_output_S2_test[0]).reshape(5000,6272)

In [ ]:
score=knn_S2.score(S2_test,y_test_KNN[:5000])

In [ ]:
scores.append(score)
scores

[0.9152, 0.8946, 0.825, 0.6792]

# Conv2D 20

In [ ]:
model.layers[2]

In [ ]:
get_conv20_output = function([model.layers[0].input],
                                  [model.layers[2].output])
layer_output_conv20 = get_conv20_output([x_train_10000])

In [ ]:
np.array(layer_output_conv20[0]).shape

In [ ]:
conv20=np.array(layer_output_conv20[0]).reshape(L,25088)

In [ ]:
knn_conv20=KNeighborsClassifier(m)
knn_conv20.fit(conv20,y_train_KNN[:L])

In [ ]:
layer_output_conv20_test = get_conv20_output([x_test[:5000]])

In [ ]:
conv20_test=np.array(layer_output_conv20_test[0]).reshape(5000,25088)

In [ ]:
score=knn_conv20.score(conv20_test,y_test_KNN[:5000])

In [ ]:
scores.append(score)
scores

# Maxpooling S1

In [ ]:
model.layers[1]

In [ ]:
get_S1_output = function([model.layers[0].input],
                                  [model.layers[1].output])
layer_output_S1 = get_S2_output([x_train_10000])

les scores [0.9142, 0.9142, 0.881, 0.8164, 0.6956, 0.653]